Checking for GPU 

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


Installing Required Packages

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-o3msmgig
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-o3msmgig
  Resolved https://github.com/huggingface/transformers to commit c612628045822f909020f7eb6784c79700813eda
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.28.0.dev0-py3-none-any.whl size=6858427 sha256=4a21b7940fdd55f16e07816d30c268566d6e1a5be46c38ff6f1fd0973fc14c60
  Stored in directory: /tmp/pip-ephem-wheel-cache-m8r98j4r/wheels/14/a0/7b/8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190999ac2b
Successfully built transformers
Looking in indexes: https://pypi.org/simp

Get your HuggingFace API Token From Login Page
- To Log Our Weights and Checkpoints

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

**DataSet Laoding**

Telugu


In [ ]:
from datasets import load_dataset, DatasetDict
# dataset = load_dataset("parambharat/telugu_asr_corpus",split='train[:10%]')


voice = DatasetDict()

# increase the size of the set  
# refer https://huggingface.co/docs/datasets/v1.11.0/splits.html


# we have to most of the data

voice["train"] = load_dataset("parambharat/telugu_asr_corpus",split='train[:10%]',use_auth_token=True)
voice["test"] = load_dataset("parambharat/telugu_asr_corpus",split='train[20%:25%]',use_auth_token=True)

print(voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'path', 'sentence', 'length'],
        num_rows: 20927
    })
    test: Dataset({
        features: ['audio', 'path', 'sentence', 'length'],
        num_rows: 10464
    })
})


In [ ]:
next(iter(voice["test"]))

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/71d29ee4a8b94d1a96d90c5642217ca18b4ae816a3291976fb5dad124916c2e0/train/KjbX4pqUQKx39P2ZtMUuz9.mp3',
  'array': array([-0.01045386, -0.01173081, -0.01189024, ...,  0.00058666,
          0.00032559,  0.00026508]),
  'sampling_rate': 16000},
 'sentence': 'నవనిర్మాణ దీక్ష పేరుతో రాష్ట్ర ప్రభుత్వం    చేసిందన్నారు'}

In [ ]:
voice = voice.remove_columns(["path", "length"])

In [ ]:
from transformers import WhisperProcessor
from transformers import WhisperTokenizer
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(
    "openai/whisper-tiny"
)
tokenizer = WhisperTokenizer.from_pretrained(
    "openai/whisper-tiny", 
     language="Telugu",
     task="transcribe",
     model_max_length=225
)
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-tiny",
     language="Telugu", 
     task="transcribe",
     model_max_length=225
)



In [ ]:
input_str = voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 అక్కడ మీడియాతో మాట్లాడిన అనంతరం తిరిగి హైదరాబాద్ బయలుదేరుతారు
Decoded w/ special:    <|startoftranscript|><|te|><|transcribe|><|notimestamps|>అక్కడ మీడియాతో మాట్లాడిన అనంతరం తిరిగి హైదరాబాద్ బయలుదేరుతారు<|endoftext|>
Decoded w/out special: అక్కడ మీడియాతో మాట్లాడిన అనంతరం తిరిగి హైదరాబాద్ బయలుదేరుతారు
Are equal:             True


In [ ]:
from datasets import Audio

voice = voice.cast_column("audio", Audio(sampling_rate=16000))


In [ ]:
import string
def fix_sentence(sentence):
    transcription = sentence
  
    if transcription.startswith('"') and transcription.endswith('"'):
        # we can remove trailing quotation marks as they do not affect the transcription
        transcription = transcription[1:-1]
  
    if transcription[-1] not in [".", "?", "!"]:
        # append a full-stop to sentences that do not end in punctuation
        transcription = transcription + "."
    transcription = transcription[:-1].translate(str.maketrans('', '', string.punctuation)) + transcription[-1]
    return transcription
    

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    sentences = fix_sentence(batch["sentence"])
    # encode target text to label ids 
    batch["labels"] = tokenizer(sentences, max_length=225, truncation=True).input_ids
    return batch


In [ ]:
voice = voice.map(prepare_dataset, remove_columns=voice.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/20927 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/10464 [00:00<?, ? examples/s]

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [ ]:
import evaluate

metric = evaluate.load("wer")


# evaluate with the 'normalised' WER
do_normalize_eval = True


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True, normalize=do_normalize_eval)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True, normalize=do_normalize_eval)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny", use_cache=False)
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False

In [ ]:
from transformers import Seq2SeqTrainingArguments


# Ideas:
"""
Use Int8 Quantize
Add Different Optimizers,per_device_train_batch_size,gradient_accumulation_steps

"""
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-te",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=voice["test"],
    eval_dataset=voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


In [ ]:
trainer.train()

In [ ]:
kwargs = {
    "dataset_tags": "parambharat/telugu_asr_corpus",
    "dataset": "telugu_asr_corpus",  # a 'pretty' name for the training dataset
    "dataset_args": "config: te, split: test",
    "language": "te",
    "model_name": "Whisper Tiny te - Eswar",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-tiny",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}


In [ ]:
trainer.push_to_hub(**kwargs)